# Linear Regression Solutions
This notebook replicates the exercises and provides working implementations and examples for: least squares (closed form) and gradient descent, both for 1D and multi-dimensional inputs.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
plt.style.use("seaborn-v0_8")

## Toy 1D dataset (same as the exercise)

In [ ]:
B0 = -2
B1 = 1.7
N = 15
X_min, X_max = 0, 5
epsilon = 5e-1

def true_func(X):
    return B0 + B1 * X

X_obs = np.random.rand(N) * (X_max-X_min) + X_min
y_obs = true_func(X=X_obs) + np.random.randn(N)*epsilon

plt.figure(figsize=(8,5))
plt.scatter(X_obs, y_obs, color='blue', s=50, label='observed')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Toy observed data')
plt.legend()
plt.show()

## 1D Least Squares (closed-form) - solution implementation

In [ ]:
def fit_linear_regression(X, y):
    """
    Fit a 1D linear regression model using the closed-form least squares solution.

    Parameters
    ----------
    X : array-like, shape (n_samples,)
        Feature values.
    y : array-like, shape (n_samples,)
        Target values.

    Returns
    -------
    beta0, beta1 : floats
        Intercept and slope.
    """
    X = np.asarray(X).ravel()
    y = np.asarray(y).ravel()
    n = X.shape[0]
    X_design = np.column_stack((np.ones(n), X))
    # Use least squares solver for stability
    beta, *_ = np.linalg.lstsq(X_design, y, rcond=None)
    return float(beta[0]), float(beta[1])

In [ ]:
# Test closed-form on toy data
beta0_est, beta1_est = fit_linear_regression(X_obs, y_obs)
print(f'Estimated parameters: β0={beta0_est:.4f}, β1={beta1_est:.4f}')
print(f'True parameters: β0={B0}, β1={B1}')

## 1D Gradient Descent - solution implementation

In [ ]:
def fit_linear_regression_gd(X, y, learning_rate=0.01, n_steps=1000):
    """
    Fit a 1D linear regression model using gradient descent.

    Parameters
    ----------
    X : array-like, shape (n_samples,)
    y : array-like, shape (n_samples,)
    learning_rate : float
    n_steps : int

    Returns
    -------
    beta0, beta1, loss_history
    """
    X = np.asarray(X).ravel()
    y = np.asarray(y).ravel()
    n = X.shape[0]
    beta0 = 0.0
    beta1 = 0.0
    loss_history = []
    for _ in range(n_steps):
        y_pred = beta0 + beta1 * X
        resid = y - y_pred
        loss = np.mean(resid**2)
        loss_history.append(loss)
        grad_b0 = -2.0/n * np.sum(resid)
        grad_b1 = -2.0/n * np.sum(resid * X)
        beta0 = beta0 - learning_rate * grad_b0
        beta1 = beta1 - learning_rate * grad_b1
    return beta0, beta1, loss_history

In [ ]:
# Test GD on toy data
b0_gd, b1_gd, loss_hist = fit_linear_regression_gd(X_obs, y_obs, learning_rate=0.05, n_steps=2000)
print(f'GD Estimated: β0={b0_gd:.4f}, β1={b1_gd:.4f}')
print(f'True: β0={B0}, β1={B1}')

plt.figure(figsize=(6,4))
plt.plot(loss_hist)
plt.title('GD loss history (toy)')
plt.xlabel('Iteration')
plt.ylabel('MSE')
plt.show()

## Multi-dimensional Least Squares - solution implementation

In [ ]:
def fit_linear_regression_multi(X, y):
    """
    Fit linear regression for multi-dimensional input using closed-form least squares.

    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
    y : array-like, shape (n_samples,)

    Returns
    -------
    beta : ndarray, shape (n_features + 1,)
        Intercept followed by feature coefficients.
    """
    X = np.asarray(X)
    y = np.asarray(y).ravel()
    n = X.shape[0]
    X_design = np.hstack((np.ones((n,1)), X))
    beta, *_ = np.linalg.lstsq(X_design, y, rcond=None)
    return beta

## Multi-dimensional Gradient Descent - solution implementation

In [ ]:
def fit_linear_regression_gd_multi(X, y, learning_rate=0.01, n_steps=1000):
    """
    Fit linear regression for multi-dimensional input using gradient descent.

    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
    y : array-like, shape (n_samples,)
    learning_rate : float
    n_steps : int

    Returns
    -------
    beta : ndarray, shape (n_features + 1,)
    loss_history : list
    """
    X = np.asarray(X)
    y = np.asarray(y).ravel()
    n = X.shape[0]
    X_design = np.hstack((np.ones((n,1)), X))
    m = X_design.shape[1]
    beta = np.zeros(m)
    loss_history = []
    for _ in range(n_steps):
        y_pred = X_design.dot(beta)
        resid = y - y_pred
        loss = np.mean(resid**2)
        loss_history.append(loss)
        grads = -2.0/n * X_design.T.dot(resid)
        beta = beta - learning_rate * grads
    return beta, loss_history

## California Housing examples: compare closed-form vs sklearn and GD

In [ ]:
# Load dataset
california = fetch_california_housing()
X_full = pd.DataFrame(california.data, columns=california.feature_names)
y = california.target

# Select features
features = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms']
X_ca = X_full[features].values

# Closed-form solution
beta_cf = fit_linear_regression_multi(X_ca, y)
print('Closed-form coefficients (intercept first):')
print(beta_cf)

# sklearn solution for comparison
lr = LinearRegression().fit(X_ca, y)
sk_coef = np.concatenate(([lr.intercept_], lr.coef_))
print('sklearn coefficients (intercept first):')
print(sk_coef)

# Predictions and metrics (closed-form)
X_design = np.hstack((np.ones((X_ca.shape[0],1)), X_ca))
y_pred_cf = X_design.dot(beta_cf)
mse_cf = mean_squared_error(y, y_pred_cf)
r2_cf = r2_score(y, y_pred_cf)
print(f'Closed-form MSE: {mse_cf:.4f}, R2: {r2_cf:.4f}')

# Gradient descent solution (multi-dim)
beta_gd, loss_hist = fit_linear_regression_gd_multi(X_ca, y, learning_rate=0.0001, n_steps=2000)
print('GD coefficients (intercept first):')
print(beta_gd)
y_pred_gd = np.hstack((np.ones((X_ca.shape[0],1)), X_ca)).dot(beta_gd)
print(f'GD MSE: {mean_squared_error(y, y_pred_gd):.4f}, R2: {r2_score(y, y_pred_gd):.4f}')

# Plot loss history for GD
plt.figure(figsize=(6,4))
plt.plot(loss_hist)
plt.title('GD loss history (California)')
plt.xlabel('Iteration')
plt.ylabel('MSE')
plt.show()

# Scatter actual vs predicted (closed-form)
plt.figure(figsize=(6,6))
plt.scatter(y, y_pred_cf, alpha=0.3)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
plt.xlabel('Actual')
plt.ylabel('Predicted (closed-form)')
plt.title('Actual vs Predicted (closed-form)')
plt.show()